<a href="https://colab.research.google.com/github/hana-dool/Pytorch/blob/master/8.Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 로지스틱 회귀

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## 직접 입력방식

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [4]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [5]:
W = torch.zeros((2, 1), requires_grad=True) # 크기는 2 x 1
b = torch.zeros(1, requires_grad=True)

In [6]:
# 둘다 같은 식이다. torch 에서는 simgmoid 를 지원해준다.
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b))) # y_hat
hypothesis = torch.sigmoid(x_train.matmul(W) + b) # 

In [7]:
# 이때에 W,b 는 모두 0 으로 초기화 되어있는 상태이다. 
# 이 상태에서 예측값을 출력해보자.
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [8]:
# loss 정의 , cost 정의
losses = -(y_train * torch.log(hypothesis) + 
           (1 - y_train) * torch.log(1 - hypothesis))
print(losses)
cost = losses.mean()
print(cost)

F.binary_cross_entropy(hypothesis, y_train) # 로 바로 계산할수도있다.

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)
tensor(0.6931, grad_fn=<MeanBackward0>)


tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

In [20]:
# 모델 변수설정
W = torch.zeros((2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True) 
# optimizer 설정
optimizer = optim.SGD([W,b], lr=1)

# epoch 설정
nb_epochs = 1000
for epoch in range(nb_epochs+1) :
    # y_hat 계산식
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis , y_train)
    # cost 로 w update
    optimizer.zero_grad() # 우선 기울기를 0 로 초기화해야한다.
    cost.backward()  # 비용함수(cost)의 기울기를 계산해서 gradient 를 계산한다.
    optimizer.step() # optimizer 에 저장한 방식으로 W,b 를 업데이트
    # log 출력
    if epoch % 100 == 0:
        print('Epoch {0:4d}/{1} Cost "{2:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost "0.693147
Epoch  100/1000 Cost "0.134722
Epoch  200/1000 Cost "0.080643
Epoch  300/1000 Cost "0.057900
Epoch  400/1000 Cost "0.045300
Epoch  500/1000 Cost "0.037261
Epoch  600/1000 Cost "0.031672
Epoch  700/1000 Cost "0.027556
Epoch  800/1000 Cost "0.024394
Epoch  900/1000 Cost "0.021888
Epoch 1000/1000 Cost "0.019852


In [30]:
# 예측
# 현재 W,b 는 requires_grad 가 변하는것을 True 로 두었으므로, 예측값이 update 되어있는 상태이다.
print(W)
print(b)
predict = torch.sigmoid(x_train.matmul(W) + b)
predict > torch.FloatTensor([0.5])  #  예측 나름 잘하는듯?

tensor([[3.2530],
        [1.5179]], requires_grad=True)
tensor([-14.4819], requires_grad=True)


tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])

## nn.Mudule 이용



In [32]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

nn.Sequential()은 nn.Module 층을 차례로 쌓을 수 있도록 합니다. 뒤에서 이를 이용해서 인공 신경망을 구현하게 되므로 기억하고 있으면 좋습니다. 조금 쉽게 말해서 nn.Sequential()은 Wx+b와 같은 수식과 시그모이드 함수 등과 같은 여러 함수들을 연결해주는 역할을 합니다. 이를 이용해서 로지스틱 회귀를 구현해봅시다.

In [34]:
model = nn.Sequential(
   nn.Linear(2, 1), # input_dim = 2, output_dim = 1
                    # Linear(2,1) 로 선언이 되어있으면, W,b 가 requres_grad = True 가 되어있는 상태로 초기화 되어서 자동으로 저장된다.
   nn.Sigmoid() ) # 출력은 시그모이드 함수를 거친다 

In [37]:
print(list(model.parameters())) # 모델이 가지고 있는 파라미터 (W,b 출력) (랜덤초기화 되어있는상태)
print(model(x_train)) # 예측값 .사실 가중치 초기화만 되어있고 학습은 안된상태라 의미는 없다.

[Parameter containing:
tensor([[ 0.3643, -0.3121]], requires_grad=True), Parameter containing:
tensor([-0.1371], requires_grad=True)]
tensor([[0.4020],
        [0.4147],
        [0.6556],
        [0.5948],
        [0.6788],
        [0.8061]], grad_fn=<SigmoidBackward>)


In [46]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    # keras 랑은 다르게 직접 지정해주어야 하는듯...
    if epoch % 100 == 0:
        prediction = (hypothesis >= torch.FloatTensor([0.5])) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = (prediction.float() == y_train) # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
                                                                             # torch 에 하나의 값만 존재한다면, item 을 통해 값을 꺼낼 수 있다.
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))


Epoch    0/1000 Cost: 0.005255 Accuracy 100.00
Epoch  100/1000 Cost: 0.005129 Accuracy 100.00
Epoch  200/1000 Cost: 0.005010 Accuracy 100.00
Epoch  300/1000 Cost: 0.004896 Accuracy 100.00
Epoch  400/1000 Cost: 0.004787 Accuracy 100.00
Epoch  500/1000 Cost: 0.004683 Accuracy 100.00
Epoch  600/1000 Cost: 0.004583 Accuracy 100.00
Epoch  700/1000 Cost: 0.004487 Accuracy 100.00
Epoch  800/1000 Cost: 0.004395 Accuracy 100.00
Epoch  900/1000 Cost: 0.004307 Accuracy 100.00
Epoch 1000/1000 Cost: 0.004223 Accuracy 100.00


## 클래스로 구현

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [50]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [51]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__() # nn.Module 의 init 을 실행한 후 거기에다가 모델 추가
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x): #  모델이 학습데이터를 입력받아서 forward 연산을 진행시키는 메서드
        return self.sigmoid(self.linear(x)) # forward 부분의 함수연산
model = BinaryClassifier()

In [52]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/1000 Cost: 0.539713 Accuracy 83.33%
Epoch  100/1000 Cost: 0.134272 Accuracy 100.00%
Epoch  200/1000 Cost: 0.080486 Accuracy 100.00%
Epoch  300/1000 Cost: 0.057820 Accuracy 100.00%
Epoch  400/1000 Cost: 0.045251 Accuracy 100.00%
Epoch  500/1000 Cost: 0.037228 Accuracy 100.00%
Epoch  600/1000 Cost: 0.031649 Accuracy 100.00%
Epoch  700/1000 Cost: 0.027538 Accuracy 100.00%
Epoch  800/1000 Cost: 0.024381 Accuracy 100.00%
Epoch  900/1000 Cost: 0.021877 Accuracy 100.00%
Epoch 1000/1000 Cost: 0.019843 Accuracy 100.00%
